# Ingestão de Dados - Oracle Database

Este notebook realiza a ingestão de dados de um banco de dados Oracle para o MinIO usando DeltaLake.

## Configuração

Configure as variáveis abaixo antes de executar:

In [ ]:
# Importar configurações base
%run ../00_configuracao_inicial.ipynb

In [ ]:
# ============================================
# CONFIGURAÇÕES DE CONEXÃO ORACLE
# ============================================
import os

# Configurações de conexão Oracle
ORACLE_HOST = os.getenv('ORACLE_HOST', 'localhost')
ORACLE_PORT = os.getenv('ORACLE_PORT', '1521')
ORACLE_SERVICE_NAME = os.getenv('ORACLE_SERVICE_NAME', 'ORCL')
ORACLE_USER = os.getenv('ORACLE_USER', 'usuario')
ORACLE_PASSWORD = os.getenv('ORACLE_PASSWORD', 'senha')

# Configurações de leitura
ORACLE_SCHEMA = os.getenv('ORACLE_SCHEMA', 'SCHEMA_NAME')
ORACLE_TABLE = os.getenv('ORACLE_TABLE', 'NOME_TABELA')

# Configurações de destino no MinIO
DESTINO_BRONZE = f"{PATH_BRONZE}/oracle/{ORACLE_SCHEMA.lower()}/{ORACLE_TABLE.lower()}"

print("Configurações Oracle:")
print(f"Host: {ORACLE_HOST}")
print(f"Port: {ORACLE_PORT}")
print(f"Service Name: {ORACLE_SERVICE_NAME}")
print(f"Schema: {ORACLE_SCHEMA}")
print(f"Table: {ORACLE_TABLE}")
print(f"Destino: {DESTINO_BRONZE}")

In [ ]:
# Instalar driver JDBC Oracle (executar apenas uma vez)
# !pip install cx_Oracle

# Adicionar driver JDBC Oracle ao Spark
# Baixar ojdbc8.jar e colocar no classpath do Spark
# Ou usar: spark.jars.packages com coordenadas Maven

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# URL de conexão JDBC Oracle
jdbc_url = f"jdbc:oracle:thin:@{ORACLE_HOST}:{ORACLE_PORT}/{ORACLE_SERVICE_NAME}"

# Propriedades de conexão
connection_properties = {
    "user": ORACLE_USER,
    "password": ORACLE_PASSWORD,
    "driver": "oracle.jdbc.OracleDriver"
}

print(f"JDBC URL: {jdbc_url}")

In [ ]:
# Função para ler dados do Oracle
def ler_oracle_table(table_name, schema=None, query=None, partition_column=None, num_partitions=None, lower_bound=None, upper_bound=None):
    """
    Lê dados de uma tabela Oracle
    
    Args:
        table_name: Nome da tabela
        schema: Schema (opcional)
        query: Query SQL customizada (opcional, substitui table_name)
        partition_column: Coluna para particionamento paralelo (opcional)
        num_partitions: Número de partições (opcional)
        lower_bound: Valor mínimo para particionamento (opcional)
        upper_bound: Valor máximo para particionamento (opcional)
    
    Returns:
        DataFrame do Spark
    """
    if query:
        # Usar query customizada (subquery)
        table_or_query = f"({query}) oracle_table"
    elif schema:
        table_or_query = f"{schema}.{table_name}"
    else:
        table_or_query = table_name
    
    reader = spark.read.format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", table_or_query) \
        .option("user", ORACLE_USER) \
        .option("password", ORACLE_PASSWORD) \
        .option("driver", "oracle.jdbc.OracleDriver")
    
    # Adicionar opções de particionamento se fornecidas
    if partition_column and num_partitions:
        reader = reader.option("partitionColumn", partition_column) \
                      .option("numPartitions", num_partitions)
        if lower_bound is not None and upper_bound is not None:
            reader = reader.option("lowerBound", lower_bound) \
                          .option("upperBound", upper_bound)
    
    df = reader.load()
    
    return df

In [ ]:
# Exemplo 1: Leitura simples de tabela
print("Exemplo 1: Leitura simples")
df_oracle = ler_oracle_table(
    table_name=ORACLE_TABLE,
    schema=ORACLE_SCHEMA
)

print(f"Total de registros: {df_oracle.count()}")
df_oracle.printSchema()
df_oracle.show(5, truncate=False)

In [ ]:
# Exemplo 2: Leitura com query customizada
print("Exemplo 2: Leitura com query customizada")
query_customizada = f"""
    SELECT 
        coluna1,
        coluna2,
        coluna3,
        TO_CHAR(data_atualizacao, 'YYYY-MM-DD HH24:MI:SS') as data_atualizacao_str
    FROM {ORACLE_SCHEMA}.{ORACLE_TABLE}
    WHERE data_atualizacao >= SYSDATE - 30
    ORDER BY data_atualizacao DESC
"""

# df_oracle_query = ler_oracle_table(query=query_customizada)
# df_oracle_query.show(5)

In [ ]:
# Exemplo 3: Leitura com particionamento paralelo (para tabelas grandes)
print("Exemplo 3: Leitura com particionamento")
# df_oracle_partitioned = ler_oracle_table(
#     table_name=ORACLE_TABLE,
#     schema=ORACLE_SCHEMA,
#     partition_column="id",  # Coluna numérica para particionamento
#     num_partitions=10,
#     lower_bound=1,
#     upper_bound=1000000
# )
# df_oracle_partitioned.show(5)

In [ ]:
# Adicionar metadados de ingestão
df_ingestao = df_oracle \
    .withColumn("fonte", lit("ORACLE")) \
    .withColumn("schema_origem", lit(ORACLE_SCHEMA)) \
    .withColumn("tabela_origem", lit(ORACLE_TABLE)) \
    .withColumn("ingestao_em", current_timestamp()) \
    .withColumn("particao_data", date_format(current_date(), "yyyy-MM-dd"))

print("Metadados adicionados:")
df_ingestao.select("fonte", "schema_origem", "tabela_origem", "ingestao_em").show(1, truncate=False)

In [ ]:
# Salvar no MinIO como Delta Table
print(f"Salvando dados em: {DESTINO_BRONZE}")

# save_delta_table(
#     df_ingestao,
#     DESTINO_BRONZE,
#     mode="overwrite",  # ou "append" para incrementais
#     partition_by=["particao_data"]  # Particionar por data
# )

print("Ingestão concluída com sucesso!")

In [ ]:
# Verificar dados salvos
# df_verificacao = read_delta_table(DESTINO_BRONZE)
# print(f"Registros salvos: {df_verificacao.count()}")
# df_verificacao.show(5)

## Ingestão Incremental

Para ingestões incrementais baseadas em timestamp ou ID:

In [ ]:
# Função para ingestão incremental
def ingestao_incremental_oracle(table_name, schema, coluna_timestamp="data_atualizacao", ultima_execucao=None):
    """
    Realiza ingestão incremental de dados Oracle
    
    Args:
        table_name: Nome da tabela
        schema: Schema
        coluna_timestamp: Nome da coluna de timestamp para filtro
        ultima_execucao: Timestamp da última execução (formato Oracle: 'YYYY-MM-DD HH24:MI:SS')
    """
    if ultima_execucao:
        query = f"""
            SELECT * FROM {schema}.{table_name}
            WHERE {coluna_timestamp} > TO_TIMESTAMP('{ultima_execucao}', 'YYYY-MM-DD HH24:MI:SS')
            ORDER BY {coluna_timestamp}
        """
    else:
        # Primeira execução: pegar últimos 7 dias
        query = f"""
            SELECT * FROM {schema}.{table_name}
            WHERE {coluna_timestamp} >= SYSDATE - 7
            ORDER BY {coluna_timestamp}
        """
    
    df_incremental = ler_oracle_table(query=query)
    
    return df_incremental

# Exemplo de uso
# df_incremental = ingestao_incremental_oracle(
#     table_name=ORACLE_TABLE,
#     schema=ORACLE_SCHEMA,
#     ultima_execucao="2024-01-01 00:00:00"
# )
# 
# # Salvar em modo append
# save_delta_table(df_incremental, DESTINO_BRONZE, mode="append", partition_by=["particao_data"])